In [7]:
from dataset import RetData, transform
from modules2 import FinTransformer
from train import trainer, tester
import torch
from utils import CHARAS_LIST
from tqdm import tqdm
from logger import Logger
import sys

import datetime

import pandas as pd
import pickle as pkl

import warnings
warnings.filterwarnings('ignore')

In [9]:
# RELOADING THE MODULES
del sys.modules['dataset']
from dataset import RetData, transform

In [ ]:
char_data = pd.read_csv("../datasets/dataset.csv")

In [16]:
with open('../datasets/month_ret.pkl', "rb") as fp:
    month_ret = pkl.load(fp)

In [19]:
month_ret_2016 = month_ret[month_ret['date']>20160101].reset_index(drop=True)
month_ret_2016 = month_ret_2016.astype('float32')
# month_ret_2016.to_pickle("../datasets/month_ret_2016.pkl")

In [20]:
month_ret_2016

,permno,date,month,ret-rf
0,10001.0,20160128.0,201601.0,11.667900
1,10025.0,20160128.0,201601.0,10.022400
2,10026.0,20160128.0,201601.0,-7.458400
3,10028.0,20160128.0,201601.0,21.202101
4,10032.0,20160128.0,201601.0,0.075900
...,...,...,...,...
68391,93427.0,20161230.0,201612.0,-5.871100
68392,93428.0,20161230.0,201612.0,-0.632400
68393,93429.0,20161230.0,201612.0,7.212400
68394,93434.0,20161230.0,201612.0,-4.196700


In [24]:
# char_data = char_data.sort_values(by=['DATE','permno']).reset_index(drop=True)  
char_data.groupby('permno').count().head(100)

,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
permno,,,,,,,,,,,,,,,,,,,,,
10000,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
10001,379,379,379,379,379,379,379,379,379,379,...,379,379,379,379,379,379,379,379,379,379
10002,325,325,325,325,325,325,325,325,325,325,...,325,325,325,325,325,325,325,325,325,325
10003,119,119,119,119,119,119,119,119,119,119,...,119,119,119,119,119,119,119,119,119,119
10005,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10099,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
10100,324,324,324,324,324,324,324,324,324,324,...,324,324,324,324,324,324,324,324,324,324
10101,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28


In [26]:
char_data_2016 = char_data[(char_data['DATE']>20160101) & \
                           (char_data['DATE']< 20161231) & \
                            (char_data['permno'] < 10103)].reset_index(drop=True).iloc[:1000,:]
char_data_2016 = char_data_2016.astype('float32')
char_data_2016.to_csv("../datasets/dataset_2016_100.csv", index=False)

In [27]:
char_data_2016

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
0,10001.0,20160128.0,7.826225e+04,0.430952,0.185720,-0.189590,10.016454,0.029586,-0.088785,-0.132151,...,0.291511,2.0,0.040133,9.489752e-08,0.149231,0.043428,0.647057,2.455863,2.749175e-08,49.0
1,10025.0,20160128.0,3.936965e+05,1.132131,1.281720,0.448376,13.642941,0.051843,0.022355,-0.153871,...,0.067424,3.0,0.043700,9.624816e-09,0.037824,0.023640,0.516746,2.911108,1.548681e-08,30.0
2,10026.0,20160128.0,2.179046e+06,0.719677,0.517934,0.036868,14.320714,0.022198,0.062914,0.003257,...,0.042928,6.0,0.017386,1.489416e-09,0.019931,0.010658,0.405442,1.993868,2.991598e-08,20.0
3,10028.0,20160128.0,4.057680e+03,0.604534,0.365461,0.372603,7.236720,0.096941,-0.109215,-0.102285,...,0.089802,2.0,0.069181,1.117250e-05,0.156250,0.052222,1.186593,1.338846,6.103379e-08,59.0
4,10032.0,20160128.0,1.161998e+06,1.534486,2.354648,-0.268983,14.230391,0.025245,-0.048157,-0.061290,...,0.080384,4.0,0.020069,2.768738e-09,0.024800,0.012676,0.492890,1.901369,2.413722e-08,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,10026.0,20161230.0,2.268677e+06,0.487005,0.237174,0.248067,14.277339,0.022772,0.099837,-0.005894,...,0.046315,6.0,0.032059,1.451460e-09,0.024094,0.013951,0.402847,1.452046,2.435477e-08,20.0
81,10028.0,20161230.0,1.176955e+04,0.379674,0.144153,0.350668,7.621367,0.108336,0.127656,0.021505,...,0.050519,2.0,0.044655,1.185150e-05,0.043956,0.026350,1.470287,1.116580,2.000000e+00,59.0
82,10032.0,20161230.0,1.712616e+06,1.410130,1.988467,-0.017166,14.052136,0.025905,0.043702,0.115477,...,0.073540,2.0,0.019855,1.047430e-09,0.029387,0.011335,0.388232,2.040013,2.039639e-08,36.0
83,10044.0,20161230.0,5.953518e+04,0.544936,0.296955,0.136008,9.791471,0.024576,0.099837,0.008824,...,0.173868,4.0,0.018442,9.247135e-08,0.027778,0.012572,0.604568,1.241434,4.706944e-08,20.0


---

In [10]:
r = RetData(returnpath = "../datasets/month_ret_2016.pkl", datapath = "../datasets/dataset_2016_100.csv")

----------In rank normalize-----------


100%|██████████| 12/12 [00:00<00:00, 94.07it/s]


(85, 3)
--------In Modify---------
---Joining Returns: 0---
---Joining Characteristics: 0---
---Joining Returns: 1---
---Joining Characteristics: 1---
---Joining Returns: 2---
---Joining Characteristics: 2---
---Joining Returns: 3---
---Joining Characteristics: 3---
---Joining Returns: 4---
---Joining Characteristics: 4---
---Joining Returns: 5---
---Joining Characteristics: 5---
---Joining Returns: 6---
---Joining Characteristics: 6---
----------In joinrets------------


In [33]:
# RELOADING THE MODULES
del sys.modules['dataset']
from dataset import RetData, transform

In [4]:
train, valid, test, df_train, df_valid, df_test = r.loaders()

-------In Loaders-------


100%|██████████| 2/2 [00:00<00:00, 669.96it/s]

-------Making tensors--------
torch.Size([8, 49])
torch.Size([4, 49])
torch.Size([2, 49])
torch.Size([8, 49])
torch.Size([4, 49])
torch.Size([2, 49])
torch.Size([8, 49])
torch.Size([4, 49])
torch.Size([2, 49])


In [6]:
df_test.to_csv("./output/Pred_test.csv", index=False)
df_train.to_csv("./output/Pred_train.csv", index=False)
df_valid.to_csv("./output/Pred_valid.csv", index=False)

In [4]:
# train, valid, test = r.loaders()

trainloader = transform(data = train, name = "trainloader.pt", batch_size =2048 )
testloader = transform(data = test, name = "testloader.pt", batch_size = 2048)
validloader = transform(data = valid, name = "validloader.pt", batch_size = 2048)

In [5]:
model = FinTransformer(embed_size=512, 
                       inner_dim = 512, 
                       num_returns=8,     #98 (company num)
                       num_decoder_returns=2, 
                       heads=8, 
                       repeats=1, dropout=0.1)

device = "cuda"
model = model.to(device)

# Training

In [6]:
# RELOADING THE MODULES
del sys.modules['modules2']
from modules2 import *

In [7]:
# Parameter setting
timestamp = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
filename = f'log_{timestamp}.txt'
epoch = 1000
lr = 1e-9   #1e-6 to 1e-9
batch_size = 2048

with Logger(filename):

    print(f'================= Epochs: {epoch}; lr: {lr}; batch_size: {batch_size} =================')

    # training
    trainer(model=model, Epochs=epoch, lr = lr, trainloader=trainloader, validloader=validloader, label = timestamp)


100%|██████████| 1/1 [00:00<00:00, 50.03it/s]


KeyboardInterrupt: 

In [18]:
# RELOADING THE MODULES
del sys.modules['train']
from train import trainer, tester


In [23]:
pred_test_res = tester(model = model, testloader=testloader,label = timestamp)

100%|██████████| 1/1 [00:00<00:00, 29.05it/s]

Test Loss = 0.9331341981887817
R2 Score = -43.50188446044922


In [ ]:
for data in testloader:
    print(len(data))

---

In [20]:
for data in testloader:
    print(len(data))

3


In [21]:
# trainer(model=model, Epochs=1000, lr = 1e-7, trainloader=trainloader, validloader=validloader)

In [22]:
char_data_2016

NameError: name 'char_data_2016' is not defined